## Day 2 Exercise
building upon the day1 exercise to offer a multi models via dropdown.
externalized the common methods into a AISystem.py file to be reused down the line

## Load env, check for api keys and load up the connections

In [1]:
import os
from enum import Enum, auto
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from AISystem import formatPrompt, AI, AISystem
import gradio as gr # oh yeah!

# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

openai = OpenAI()

claude = anthropic.Anthropic()

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
openai_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model = "gemini-1.5-flash"
ollama_model = "llama3.2"

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyB9


## Create the systems for each llms

In [2]:
geminiSys = AISystem(gemini_via_openai_client,
                        formatPrompt("system","You are a chatbot. you always try to make conversation and get more in depth"), 
                        gemini_model,
                        AI.GEMINI)

openAiSys = AISystem(openai,
                        formatPrompt("system","You are a chatbot. you always try to make conversation and get more in depth"), 
                        openai_model,
                        AI.OPEN_AI)

claudeSys = AISystem(claude,
                       "You are a chatbot. you always try to make conversation and get more in depth", 
                       claude_model,
                        AI.CLAUDE)

ollamaSys = AISystem(ollama_via_openai,
                        formatPrompt("system","You are a chatbot. you always try to make conversation and get more in depth"), 
                        ollama_model,
                        AI.OLLAMA)
sys_dict = { AI.GEMINI: geminiSys, AI.OPEN_AI: openAiSys, AI.CLAUDE: claudeSys, AI.OLLAMA: ollamaSys}

def stream_model(prompt, model):
    aiSystem = sys_dict.get(AI[model.upper()])
    yield from aiSystem.stream(formatPrompt("user",prompt), True)

## Create the gradio interface linking with the AI enum for the dropdown

In [3]:

view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your prompt:", lines=6) , gr.Dropdown(choices=[ai.value for ai in AI], label="Select model")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/httpx/_transports/default.py", line 72, in map_httpcore_exceptions
    yield
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/httpx/_transports/default.py", line 236, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/httpcore/_sync/connection_pool.py", line 256, in handle_request
    raise exc from None
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/httpcore/_sync/connection_pool.py", line 236, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/httpcore/_sync/connection.py", line 101, in handle_request
    raise exc
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/httpcore/_sync/connection.py", line 78, in handle_request
    st